In [1]:
from keras import models, layers
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from datagenerator import DataGenerator
import datetime
import pickle
from pyntcloud import PyntCloud
from mpl_toolkits.mplot3d import Axes3D
import modelutils

/Users/tristanbehrens/Development/python-venvs/venv-3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Hook to the proper dataset.

In [2]:
if os.path.exists("datasetpath.txt"):
    dataset_path = open("datasetpath.txt", "r").read().replace("\n", "")
else:
    dataset_path = "../data"

# Create a data-generator.

In [3]:
data_generator = DataGenerator(dataset_path=dataset_path, input_type="voxelgrid", output_targets=["height", "weight"])

print("jpg_paths", len(data_generator.jpg_paths))
print("pcd_paths", len(data_generator.pcd_paths))
print("json_paths_personal", len(data_generator.json_paths_personal))
print("json_paths_measures", len(data_generator.json_paths_measures))
print("QR-Codes:\n" + "\n".join(data_generator.qrcodes))

print("Done.")

WARNING! Multiple manual measurements for QR-code: SAM-GOV-049 ../data/db/persons/c79b708e301d9c71_shaikh_1531300004131_40Aq4RZQ3gUZts74/measures/c79b708e301d9c71_measure_1531300841811_Lhp0PzvwnmlyRspG/c79b708e301d9c71_measure_1531300841811_Lhp0PzvwnmlyRspG.json
WARNING! Multiple manual measurements for QR-code: SAM-SNG-014 ../data/db/persons/f2467a1fb0c542b9_kamble_1530687006027_AVA2AqQDcVsHYrrK/measures/f2467a1fb0c542b9_measure_1531291525951_BhMYXh4Zeu9Jk7Zr/f2467a1fb0c542b9_measure_1531291525951_BhMYXh4Zeu9Jk7Zr.json
WARNING! Multiple manual measurements for QR-code: SAM-SNG-096 ../data/db/persons/f2467a1fb0c542b9_kamble_1530687464869_e6puVVsetdSpP7sH/measures/f2467a1fb0c542b9_measure_1531291761123_0ZS7kMnJLy70pe4I/f2467a1fb0c542b9_measure_1531291761123_0ZS7kMnJLy70pe4I.json
jpg_paths 7076
pcd_paths 2129
json_paths_personal 62
json_paths_measures 110
QR-Codes:
SAM-GOV-001
SAM-GOV-002
SAM-GOV-004
SAM-GOV-005
SAM-GOV-008
SAM-GOV-011
SAM-GOV-012
SAM-GOV-013
SAM-GOV-014
SAM-GOV-023
SAM-

# Do the train-validate-split.

In [4]:
# Ensure that the split is always the same.
random.seed(108)

# Filter and shuffle the QR-Codes.
qrcodes_shuffle = list(data_generator.qrcodes)
qrcodes_shuffle = [qrcode for qrcode in qrcodes_shuffle if qrcode.startswith("SAM-GOV")]
random.shuffle(qrcodes_shuffle)

# Do the split.
split_index = int(0.8 * len(qrcodes_shuffle))
qrcodes_train = qrcodes_shuffle[:split_index]
qrcodes_validate = qrcodes_shuffle[split_index:]
del qrcodes_shuffle

# Print statistics.
print("QR-Codes train:")
print(" ".join(qrcodes_train))
print("")
print("QR-Codes validate:")
print(" ".join(qrcodes_validate))
print("")
print("Done.")

QR-Codes train:
SAM-GOV-023 SAM-GOV-041 SAM-GOV-033 SAM-GOV-037 SAM-GOV-012 SAM-GOV-099 SAM-GOV-001 SAM-GOV-035 SAM-GOV-002 SAM-GOV-038 SAM-GOV-008 SAM-GOV-025 SAM-GOV-034 SAM-GOV-036 SAM-GOV-004 SAM-GOV-014 SAM-GOV-043 SAM-GOV-011 SAM-GOV-042

QR-Codes validate:
SAM-GOV-013 SAM-GOV-026 SAM-GOV-003 SAM-GOV-044 SAM-GOV-005

Done.


# Train several nets.

In [4]:
# Parameters.
batch_size = 32
step_per_epoch = 32
epochs = 10
validation_steps = 8


# Create the models.
models_to_train = []
models_to_train.append(modelutils.create_dense_model(data_generator.get_input_shape(), data_generator.get_output_size()))
models_to_train.append(modelutils.create_voxnet_model_small(data_generator.get_input_shape(), data_generator.get_output_size()))
models_to_train.append(modelutils.create_voxnet_model_big(data_generator.get_input_shape(), data_generator.get_output_size()))
models_to_train.append(modelutils.create_voxnet_model_homepage(data_generator.get_input_shape(), data_generator.get_output_size()))

# Train the models.
histories = []
for model in models_to_train:

    # Some output.
    print("Training:", model.name)
    model.summary()

    # Compile the model.
    model.compile(
            optimizer="rmsprop",
            loss="mse",
            metrics=["mae"]
        )
    
    # Train the model.
    history = model.fit_generator(
        data_generator.generate(size=batch_size, qrcodes_to_use=qrcodes_train),
        steps_per_epoch=step_per_epoch,
        epochs=epochs,
        validation_data=data_generator.generate(size=batch_size, qrcodes_to_use=qrcodes_validate),
        validation_steps=validation_steps)
    
    histories.append(history)
    print("Done.")
    print("")

Training: baseline-dense
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               4194432   
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 4,202,818
Trainable params: 4,202,818
Non-trainable params: 0
_________________________________________________________________


NameError: name 'qrcodes_train' is not defined

# Save the models.

In [ ]:
datetime_string = datetime.datetime.now().strftime("%Y%m%d-%H%M")

# Save the models and histories.    
for model in models_to_train:
    model_name = datetime_string + "-" + model.name
    model_path = os.path.join(model_name)
    #model.save(model_path + ".h5")
    #pickle.dump(history, open(model_path + ".p", "wb"))
    print("Model saved to:", model_path)
    
print("All saved.")

# Render histories.

In [ ]:
for index, history in histories:
    model = models_to_train[i]
    for key, data in history.history.items():
        plt.plot(data, label=model.name + "-" + key)
plt.show()
plt.close()